<a href="https://colab.research.google.com/github/iolef/Sarcasm-identification-in-implicit-misogyny/blob/main/1_1_Implicit_hate_detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Implicit hate detection model: three-categories classification**




In this notebook, [DistilBERT](https://https://huggingface.co/docs/transformers/model_doc/distilbert) is used to fine-tune an implicit hate detection model using the [Implicit hate corpus](https://https://github.com/SALT-NLP/implicit-hate) by El Sherief et al. (2021).

# **1. Setup**

1.1 Installing Transformers

In [ ]:
# Transformers installation
! pip install transformers[torch] datasets evaluate
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00


1.2 Imports

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd
import zipfile
import os
import re

# **2. Corpus upload**

Uploading and unzipping the corpus folder and creating a dataframe with the selected file.

In [ ]:
# Unzipping the corpus folder
with zipfile.ZipFile("implicit-hate-corpus-nov-2021.zip", "r") as zip_ref:
    zip_ref.extractall()

# Seelcting the desired subfolder from the corpus folder
dataset_path = os.path.join(os.getcwd(), "implicit-hate-corpus", "implicit_hate_v1_stg1_posts.tsv")

# Creating the dataframe
df = pd.read_csv(dataset_path, delimiter="\t")

# **3. Preprocessing**

Creating a function which includes all the text preprocessing operations.

In [ ]:
def clean_text(post):
 # Lowercasing
 post = post.lower()
 # Hashtag removal
 post = re.sub(r'((?<=[\s\W])|^)[#](\w+|[^#]|$)', ' ', post)
 # "rt" removal
 post = re.sub('^\s*rt\s+', '', post)
 # special characters removal
 post = re.sub(r'[^\w]', ' ', post)
 # stripping
 post = post.strip()
 # removing the unnecessary whitespaces between words
 post = ' '.join(post.split())
 return post

# Applying to each post the clean_text function
df["post"] = df["post"].apply(clean_text)

# **4. Preparing the data for the training**

4.1 Converting the dataframe in a format compatible with Huggingface

In [ ]:
# Renaming the columns
df.rename(columns={"post": "text", "class": "label"}, inplace=True)

# Converting the labels into numbers
df['label'] = df['label'].replace({"explicit_hate": 0, "implicit_hate": 1, "not_hate": 2})

4.2 Splitting the data

In [ ]:
# Adding a seed in order to be able to replicate the same experiment
seed = 100
# Retaining 80% of the data for the training set
train_set, test_val_set = train_test_split(df, test_size=0.2, random_state = seed)

# Dividing the remaining 20%
test_set, val_set = train_test_split(test_val_set, test_size=0.5, random_state = seed)

# Trasforming the dataframes into datasets
train_ds = Dataset.from_pandas(train_set, split="train")
val_ds = Dataset.from_pandas(val_set, split="val")
test_ds = Dataset.from_pandas(test_set, split="test")

4.3 Creating the tokenizer

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train_set = train_ds.map(preprocess_function, batched=True)
tokenized_val_set = val_ds.map(preprocess_function, batched=True)
tokenized_test_set = test_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/17184 [00:00<?, ? examples/s]

Map:   0%|          | 0/2148 [00:00<?, ? examples/s]

Map:   0%|          | 0/2148 [00:00<?, ? examples/s]

In [ ]:
# Printing a few examples
train_ds[0:100]

{'text': ['white men',
  'the war on isn t waged by islam but by the us government against its people',
  'white nationalists advocating for personalized communities sounds like him crow i would say white identity politics is history',
  'the war on white people is global they are attacking flags and statues all over the west and mass murder of whites will follow',
  'good time to issue a support white police leaflet sticker and posters',
  'and if some horny lesbian watches you pee you re in with it not disgusting at all duh',
  'white people never rape good point',
  'fair leaders have ties to white supremacist groups and eugenicists and have made many racist statements via',
  'truth about the holocaust religion terrifies its priests a nicely condensed piece on the fictional holocaust',
  'jihadis never ever abandon caliphate be not fooled they take a long term view',
  'stay white friend stay white hopefully your uncle ruckus will come for the holidays',
  '1 historically the south

# **5. Training the model**

Training the model with DistilBERT pre-trained model.

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

import evaluate

accuracy = evaluate.load("accuracy")

import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

id2label = {0: "explicit_hate", 1: "implicit_hate", 2: "not_hate"}
label2id = {"explicit_hate": 0, "implicit_hate": 1, "not_hate": 2}

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id
)

training_args = TrainingArguments(
    output_dir="my_hate_model",
    learning_rate=3e-6,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_set,
    eval_dataset=tokenized_test_set,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.757541,0.639665
2,No log,0.676089,0.714153
3,No log,0.648620,0.725791
4,0.755000,0.634292,0.727188
5,0.755000,0.626481,0.729516
6,0.755000,0.620873,0.735568
7,0.755000,0.617875,0.729981
8,0.631900,0.617750,0.732775
9,0.631900,0.618677,0.728119
10,0.631900,0.614067,0.734171


TrainOutput(global_step=2700, training_loss=0.6070093734176071, metrics={'train_runtime': 1631.5043, 'train_samples_per_second': 210.652, 'train_steps_per_second': 1.655, 'total_flos': 5102967016726272.0, 'train_loss': 0.6070093734176071, 'epoch': 20.0})

# **6. Testing the model**

Trying the model on a sample sentence.

In [ ]:
from transformers import pipeline

text = "I think black people are all bad."

classifier = pipeline(task= 'sentiment-analysis',
                      model= "my_hate_model/checkpoint-1620",
                      tokenizer = "my_hate_model/checkpoint-1620")

classifier(text)

[{'label': 'implicit_hate', 'score': 0.6357327103614807}]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
